# Model Time Series ML to predict metro frequentation during JO period with JO of stations rattached to 1 site without known past events 

In [42]:
station = "BERCY"
site = "ARENA BERCY"

In [43]:
import pandas as pd
import numpy as np

In [44]:
def events(site):
    if site == "ROLAND-GARROS":
        return "EVENT_RG"
    elif site == "PARC DES PRINCES":
        return "EVENT_PDP"
    elif site == "STADE DE FRANCE":
        return "EVENT_SDF"
    elif site == "GRAND PALAIS":
        return "EVENT_GP"
    elif site == "ARENA INVALIDES":
        return "EVENT_INV"
    elif site == "STADE DE LA CONCORDE":
        return "EVENT_CCD"
    elif site == "ARENA CHAMP DE MARS":
        return "EVENT_CDM"
    elif site == "STADE TOUR EIFFEL":
        return "EVENT_TEF"
    elif site == "ARENA PARIS SUD":
        return "EVENT_SUD"
    elif site == "ARENA LA CHAPELLE":
        return "EVENT_CHP"
    elif site == "ARENA BERCY":
        return "EVENT_BER"
    else:
        return 0

In [45]:
data=pd.read_csv(f"98-Forecast/forecast_ssjo_{station}.csv")
data.head()

,ds,yhat,station
0,2024-07-01,9.261767,BERCY
1,2024-07-02,9.225904,BERCY
2,2024-07-03,9.227383,BERCY
3,2024-07-04,9.235339,BERCY
4,2024-07-05,9.232968,BERCY


In [46]:
geoloc=pd.read_csv("99-Data_Clean/geoloc_jo24.csv")
geoloc.head()

,lieu,type,lat,lng,capacite
0,STADE DE FRANCE-SAINT-DENIS,station,48.917812,2.351067,19636.0
1,SAINT-DENIS-PORTE DE PARIS,station,48.929499,2.356273,13758.0
2,LA PLAINE-STADE DE FRANCE,station,48.918107,2.362326,22415.0
3,BOULEVARD VICTOR,station,48.839364,2.270485,4430.0
4,PORTE D'AUTEUIL,station,48.847677,2.257985,1968.0


In [47]:
df_coeff = pd.read_csv("99-Data_Clean/coeff_stations_ssevent.csv")
df_coeff.head()

,lieu,site model,station model,EVENT,EVENT_LOG
0,MUSEE D'ORSAY,ROLAND-GARROS,PORTE D'AUTEUIL,-11.520813,1.169022
1,SEGUR,ROLAND-GARROS,PORTE D'AUTEUIL,-11.520813,1.169022
2,PONT DE L'ALMA,ROLAND-GARROS,PORTE D'AUTEUIL,-11.520813,1.169022
3,CHAMP DE MARS-TOUR EIFFEL,ROLAND-GARROS,PORTE D'AUTEUIL,-11.520813,1.169022
4,PORTE DE LA CHAPELLE,ROLAND-GARROS,PORTE DE SAINT-CLOUD,1.285228,-0.113329


In [48]:
event=pd.read_csv("99-Data_Clean/event_jo.csv")
event.head()

,ds,EVENT_PDP,EVENT_SDF,EVENT_GP,EVENT_INV,EVENT_CCD,EVENT_YDM,EVENT_CDM,EVENT_TEF,EVENT_RG,EVENT_SUD,EVENT_CHP,EVENT_BER
0,2024-07-01,0,0,0,0,0,0,0,0,0,0,0,0
1,2024-07-02,0,0,0,0,0,0,0,0,0,0,0,0
2,2024-07-03,0,0,0,0,0,0,0,0,0,0,0,0
3,2024-07-04,0,0,0,0,0,0,0,0,0,0,0,0
4,2024-07-05,0,0,0,0,0,0,0,0,0,0,0,0


In [49]:
event[['ds', events(site)]]

,ds,EVENT_BER
0,2024-07-01,0
1,2024-07-02,0
2,2024-07-03,0
3,2024-07-04,0
4,2024-07-05,0
...,...,...
57,2024-08-27,0
58,2024-08-28,0
59,2024-08-29,0
60,2024-08-30,0


In [50]:
data=data.merge(event[["ds",events(site)]],on="ds")

In [51]:
cap = 0
for i in range(len(geoloc['lieu'])):
    if site == geoloc['lieu'][i]:
        cap = geoloc['capacite'][i]
    else:
        pass

In [52]:
data['cap']=data[events(site)].apply(lambda x:1 if x==0 else cap)
data.head()

,ds,yhat,station,EVENT_BER,cap
0,2024-07-01,9.261767,BERCY,0,1.0
1,2024-07-02,9.225904,BERCY,0,1.0
2,2024-07-03,9.227383,BERCY,0,1.0
3,2024-07-04,9.235339,BERCY,0,1.0
4,2024-07-05,9.232968,BERCY,0,1.0


In [53]:
coeff_1 = float(df_coeff[(df_coeff['lieu'] == station)]['EVENT'].values)
coeff_2 = float(df_coeff[(df_coeff['lieu'] == station)]['EVENT_LOG'].values)

In [54]:
data['yhatjo']=data['yhat']+(coeff_1*data[events(site)])+(coeff_2*np.log(data['cap']))

In [55]:
data[25:40]

,ds,yhat,station,EVENT_BER,cap,yhatjo
25,2024-07-26,9.221146,BERCY,0,1.0,9.221146
26,2024-07-27,8.979105,BERCY,1,15000.0,9.174582
27,2024-07-28,9.058460,BERCY,1,15000.0,9.253936
28,2024-07-29,9.217511,BERCY,1,15000.0,9.412988
29,2024-07-30,9.192262,BERCY,1,15000.0,9.387738
30,2024-07-31,9.203366,BERCY,1,15000.0,9.398842
31,2024-08-01,9.219670,BERCY,1,15000.0,9.415146
32,2024-08-02,9.224116,BERCY,1,15000.0,9.419593
33,2024-08-03,8.972919,BERCY,1,15000.0,9.168395
34,2024-08-04,9.042705,BERCY,1,15000.0,9.238181


In [56]:
data_fin = pd.concat([data['ds'],data['yhatjo'],data['station']], axis=1, ignore_index=True)
data_fin.columns = ['ds', 'yhat', 'station']
data_fin

,ds,yhat,station
0,2024-07-01,9.261767,BERCY
1,2024-07-02,9.225904,BERCY
2,2024-07-03,9.227383,BERCY
3,2024-07-04,9.235339,BERCY
4,2024-07-05,9.232968,BERCY
...,...,...,...
57,2024-08-27,9.069768,BERCY
58,2024-08-28,9.093211,BERCY
59,2024-08-29,9.124338,BERCY
60,2024-08-30,9.145882,BERCY


In [57]:
data_fin.to_csv(f"98-Forecast/forecast_jo_{station}.csv", index=False)